In [ ]:
!kill -9 -1

In [1]:
# !pip install --quiet transformers==4.5.0
# !pip install --quiet torchtext
# !pip install --quiet torchvision==0.13.0
# !pip install --quiet pytorch-lightning
# !pip install --quiet torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
# !pip install --quiet datasets
# !pip install --quiet pickle5
# !pip install --quiet rouge

# pip install tensorboard_logger
# pip install torchtext==0.13.0
# pip install numpy==1.21
# pip install torch --upgrade
# pip install torch-utils
# pip install pandas
# pip install pytorch-lightning==1.9.1
# pip install datasets
# pip install torchvision==0.12.0
# pip install transformers
# pip install packaging==21.3

!pip install --quiet packaging==21.3
!pip install --quiet transformers
!pip install --quiet torchvision==0.12.0
!pip install --quiet torchtext==0.12
!pip install --quiet pytorch-lightning==1.8
# !pip install --quiet torch==1.11.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install torch==1.13.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html
!pip install --quiet datasets
!pip install --quiet pickle5
!pip install --upgrade --quiet pandas==1.3.4
!pip install --quiet rouge
!pip install --quiet accelerate
!pip install --quiet bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

saved_model_path = "/content/drive/MyDrive/Projects/Commonsense_Reasoning/saved_model"
data_path = "/content/drive/MyDrive/Projects/Commonsense_Reasoning/data"

Mounted at /content/drive


In [ ]:
import os
import json
import time
import pandas as pd
import numpy as np
import torch
import pickle5 as pickle
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger


from termcolor import colored
import textwrap
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
)
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

print(f"torch version : {torch.__version__}")

import pytorch_lightning as pl
print(f"pytorch lightning version : {pl.__version__}")

import torchtext
print(f"torchtext version : {torchtext.__version__}")

In [ ]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

# device = torch.device('cpu')
device

In [ ]:
pl.seed_everything(42)

# **Import (custom) Squad Dataset**

In [ ]:
import pickle5 as pickle

with open(os.path.join(data_path, "squad_dataset.pkl"), "rb") as f:
  squad_dataset = pickle.load(f)

In [ ]:
train_df = pd.DataFrame({"passage": squad_dataset['train_articles'], "question": squad_dataset['train_questions'], "answer": squad_dataset['train_answers']})
train_df.answer = train_df.answer.apply(lambda x: list(x)[0])
train_df.reset_index(drop=True, inplace=True)
print(len(train_df))
train_df.head()

87599


,passage,question,answer
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary


In [ ]:
validation_df = pd.DataFrame({"passage": squad_dataset['validation_articles'][:10569], "question": squad_dataset['validation_questions'][:10569], "answer": squad_dataset['validation_answers'][:10569]})
validation_df.answer = validation_df.answer.apply(lambda x: list(x)[0])
validation_df.reset_index(drop=True, inplace=True)
print(len(validation_df))
validation_df.head()

10569


,passage,question,answer
0,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,Denver Broncos
1,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,Carolina Panthers
2,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,Levi's Stadium
3,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,Denver Broncos
4,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,gold


# **Import CookDial Dataset**

In [ ]:
recipeqa_df = pd.read_pickle(os.path.join(data_path, "recipe_qa.pickle"))
print(len(recipeqa_df))
recipeqa_df.head()

4473


,passage,question,answer
0,Title : English Walnut Pie.\t0. Ingredient : 3...,Hi. What's the first step I need to take?,Could you preheat your oven to 400 degrees?
1,Title : English Walnut Pie.\t0. Ingredient : 3...,Are the degrees in Celsius?,"No, it's 400 degrees Fahrenheit and 205 degree..."
2,Title : English Walnut Pie.\t0. Ingredient : 3...,Ok. thank you. What's the next instruction?,Take a large bowl and beat 3 eggs.
3,Title : English Walnut Pie.\t0. Ingredient : 3...,Ok. I did it. What's next?,"Now mix in 3/4 cup of white sugar, 2 teaspoons..."
4,Title : English Walnut Pie.\t0. Ingredient : 3...,Could you repeat the last ingredient please?,3/4 cup of light corn syrup which you may add ...


In [ ]:
train_df, validation_df = train_test_split(recipeqa_df, test_size=0.1, shuffle=True, random_state=42)
len(train_df), len(validation_df)

(4025, 448)

# **Import Custom Dataset**

In [ ]:
with open(os.path.join(data_path, "tacobot_v1_combined_train_test.pickle"), "rb") as f:
    data = pickle.load(f)

train_df = pd.DataFrame.from_dict(data["train"])
validation_df = pd.DataFrame.from_dict(data["test"])
len(train_df), len(validation_df)

(20405, 5098)

In [7]:
from sklearn.model_selection import train_test_split

def convert_to_df(dataset):
    data_dict = {"passage": [], "question": [], "answer": []}
    for i, datum in enumerate(dataset):
        qa_list = datum["qa"]
        for qa in qa_list:
            data_dict["passage"].append(datum["context"])
            data_dict["question"].append(qa["question"])
            data_dict["answer"].append(qa["answer"])
    df = pd.DataFrame.from_dict(data_dict)
    return df, data_dict

with open(os.path.join(data_path, "recipe_qa_augmented_dataset.pickle"), "rb") as f:
    data = pickle.load(f)

df, _ = convert_to_df(data)
train_df, validation_df = train_test_split(df, test_size=0.1, shuffle=True, random_state=42)
len(train_df), len(validation_df)

(827430, 91937)

# **Preprocess Text**

In [8]:
# replacing many abbreviations and lower casing the words
import re
import unicodedata


def clean_words(sentence):
    sentence = str(sentence).lower()
    sentence = unicodedata.normalize('NFKD', sentence).encode('ascii', 'ignore').decode('utf-8', 'ignore') # for converting é to e and other accented chars
    sentence = re.sub(r"http\S+","",sentence)
    sentence = re.sub(r"there's", "there is", sentence)
    sentence = re.sub(r"i'm", "i am", sentence)
    sentence = re.sub(r"he's", "he is", sentence)
    sentence = re.sub(r"she's", "she is", sentence)
    sentence = re.sub(r"it's", "it is", sentence)
    sentence = re.sub(r"that's", "that is", sentence)
    sentence = re.sub(r"what's", "that is", sentence)
    sentence = re.sub(r"where's", "where is", sentence)
    sentence = re.sub(r"how's", "how is", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"won't", "will not", sentence)
    sentence = re.sub(r"can't", "cannot", sentence)
    sentence = re.sub(r"n't", " not", sentence)
    sentence = re.sub(r"n'", "ng", sentence)
    sentence = re.sub(r"'bout", "about", sentence)
    sentence = re.sub(r"'til", "until", sentence)
    sentence = re.sub(r"\"", "", sentence)
    sentence = re.sub(r"\'", "", sentence)
    sentence = re.sub(r' s ', "",sentence)
    # sentence = re.sub(r"&39", "", sentence) # the inshorts data has this in it
    # sentence = re.sub(r"&34", "", sentence) # the inshorts data has this in it
    # sentence = re.sub(r"[\[\]\\0-9()\"$#%/@;:<>{}`+=~|.!?,-]", "", sentence)
    # sentence = re.sub(r"&", "", sentence)
    sentence = re.sub(r"\\n", "", sentence)
    sentence = sentence.strip()
    return sentence

In [9]:
dir_name = os.path.join(data_path, "preprocessed_data_files")
if os.path.isdir(dir_name):
  if not os.listdir(dir_name):
    # Directory is empty
    print("Preprocessing data...\n")
    train_df["passage"] = train_df["passage"].apply(lambda x: clean_words(x))
    train_df["question"] = train_df["question"].apply(lambda x: clean_words(x))
    train_df["answer"] = train_df["answer"].apply(lambda x: clean_words(x))
    train_df.to_pickle(os.path.join(data_path, "preprocessed_data_files", "train_df.pickle"))
    validation_df["passage"] = validation_df["passage"].apply(lambda x: clean_words(x))
    validation_df["question"] = validation_df["question"].apply(lambda x: clean_words(x))
    validation_df["answer"] = validation_df["answer"].apply(lambda x: clean_words(x))
    validation_df.to_pickle(os.path.join(data_path, "preprocessed_data_files", "validation_df.pickle"))
    print("Data preprocessed and saved to drive!")
  else:    
    # Directory is not empty
    print("Fetching preprocessed data from drive...\n")
    train_df = pd.read_pickle(os.path.join(data_path, "preprocessed_data_files", "train_df.pickle"))
    validation_df = pd.read_pickle(os.path.join(data_path, "preprocessed_data_files", "validation_df.pickle"))
    print("Fetched preprocessed data from drive!")
else:
  print("Invalid path! Given directory doesn't exist")

Preprocessing data...

Data preprocessed and saved to drive!


In [10]:
MODEL_NAME = "google/flan-t5-large"
# MODEL_NAME = "google/flan-t5-xl"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

**Word Count Inspection**

In [11]:
passage_token_counts, question_token_counts, answer_token_counts = [], [], []

for _, row in train_df.iterrows():
  text_token_count = len(tokenizer.encode(row["passage"]))
  passage_token_counts.append(text_token_count)

  question_token_count = len(tokenizer.encode(row["question"]))
  question_token_counts.append(question_token_count)

  answer_token_count = len(tokenizer.encode(row["answer"]))
  answer_token_counts.append(answer_token_count)

Token indices sequence length is longer than the specified maximum sequence length for this model (872 > 512). Running this sequence through the model will result in indexing errors


In [12]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)

sns.histplot(passage_token_counts, ax=ax1)
ax1.set_title("passage token counts")

sns.histplot(question_token_counts, ax=ax2)
ax2.set_title("question token counts")

sns.histplot(answer_token_counts, ax=ax3)
ax3.set_title("answer token counts")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'plt' is not defined

In [ ]:
class QADataset(Dataset):
  def __init__(
      self,
      data: pd.DataFrame,
      tokenizer: T5Tokenizer,
      source_max_token_len: int = 1024,
      target_max_token_len: int = 64
  ):
    super().__init__()
    self.data = data
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]
    source_encoding = tokenizer(
        data_row["question"],
        data_row["passage"],
        max_length=self.source_max_token_len,
        padding="max_length",
        truncation="only_second",
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
    )

    target_encoding = tokenizer(
        data_row["answer"],
        max_length=self.target_max_token_len,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
    )

    labels = target_encoding["input_ids"]
    labels[labels==0] = -100

    return dict(
        question=data_row["question"],
        context=data_row["passage"],
        answer=data_row["answer"],
        input_ids=source_encoding["input_ids"].flatten(),
        attention_mask=source_encoding["attention_mask"].flatten(),
        labels=labels.flatten()
    )    

In [ ]:
sample_dataset = QADataset(train_df, tokenizer)

In [ ]:
for data in sample_dataset:
  print(data["question"])
  print(data["input_ids"][:10])
  print(data["labels"][:10])
  break

In [ ]:
class QADataModule(pl.LightningDataModule):
  def __init__(
      self,
      train_df: pd.DataFrame,
      validation_df: pd.DataFrame,
      tokenizer: AutoTokenizer,
      batch_size: int = 8,
      source_max_token_len: int = 1024,
      target_max_token_len: int = 64
  ):
    super().__init__()
    self.train_df = train_df
    self.validation_df = validation_df
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def setup(self, stage=None):
    self.train_dataset = QADataset(
        self.train_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
    )

    self.validation_dataset = QADataset(
        self.validation_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
    )
  
  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True,
        num_workers=4
    )

  def val_dataloader(self):
    return DataLoader(
        self.validation_dataset,
        batch_size=self.batch_size,
        num_workers=4
    )

In [ ]:
BATCH_SIZE = 3
data_module = QADataModule(train_df, validation_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

In [ ]:
class QAModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    # self.model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, device_map="auto", load_in_8bit=True, return_dict=True)
    # self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, device_map='auto')
    self.model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, return_dict=True)

  def forward(self, input_ids, attention_mask, labels=None):
    output=self.model(
      input_ids=input_ids,
      attention_mask=attention_mask,
      labels=labels
    )
    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return loss

  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def configure_optimizers(self):
    return AdamW(self.parameters(), lr=0.0001)

In [ ]:
model = QAModel()

model = QAModel.load_from_checkpoint(
    os.path.join(saved_model_path, "best-checkpoint-v3.ckpt")
)

In [ ]:
def freeze_layers(model, num_encoder_unfreeze=None, num_decoder_unfreeze=None):
  modules_to_freeze = []

  if num_encoder_unfreeze:
      # Encoder modules, which has SelfAttention (layer[0])
      modules_to_freeze.extend([model.encoder.block[i].layer[0] for i in
                            range(len(model.encoder.block) - num_encoder_unfreeze)])

      # and DenseReluDense block (layer[1])
      modules_to_freeze.extend(
          [model.encoder.block[i].layer[1] for i in range(len(model.encoder.block) - num_encoder_unfreeze)])

  if num_decoder_unfreeze:
      # And the decoder modules, which has SelfAttention (layer[0])
      modules_to_freeze.extend(
          [model.decoder.block[i].layer[0] for i in range(len(model.decoder.block) - num_decoder_unfreeze)])

      # and CrossAttention block (layer[1])
      modules_to_freeze.extend(
          [model.decoder.block[i].layer[1] for i in range(len(model.decoder.block) - num_decoder_unfreeze)])

      # and DenseReluDense block (layer[2])
      modules_to_freeze.extend(
          [model.decoder.block[i].layer[2] for i in range(len(model.decoder.block) - num_decoder_unfreeze)])

  if num_decoder_unfreeze or num_decoder_unfreeze:
      for module in modules_to_freeze:
          for param in module.parameters():
              param.requires_grad = False

  if num_decoder_unfreeze or num_decoder_unfreeze:
    for module in modules_to_freeze:
        for param in module.parameters():
            param.requires_grad = False
  
  return model

In [ ]:
# from pl.callbacks.early_stopping import EarlyStopping

NUM_EPOCHS = 40

model.model = freeze_layers(model.model, num_encoder_unfreeze=None, num_decoder_unfreeze=20)

# early_stop_callback = EarlyStopping(monitor="val_accuracy", min_delta=0.00, patience=3, verbose=False, mode="max")

checkpoint_callback = ModelCheckpoint(
    dirpath=saved_model_path,
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

logger = TensorBoardLogger("lightning_logs", name="question_answering")

trainer = pl.Trainer(
    logger=logger,
    callbacks=[checkpoint_callback],
    max_epochs=NUM_EPOCHS,
    gpus=1
)

In [ ]:
for i, param in enumerate(model.model.parameters()):
    print(i, "\t", param.requires_grad)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

In [ ]:
# function ClickConnect(){
#     console.log("Clicked on connect button");
#     document.querySelector("colab-connect-button").click() 
#  }
#  setInterval(ClickConnect,60000)

In [ ]:
trainer.fit(model, data_module)

In [ ]:
checkpoint_callback.best_model_path

In [ ]:
model = QAModel.load_from_checkpoint(
    os.path.join(saved_model_path, "best-checkpoint-v3.ckpt")
)

model.to(device)
model.freeze()

In [ ]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [ ]:
print("Optimizer's state_dict:")
for var_name in model.configure_optimizers().state_dict():
    print(var_name, "\t", model.configure_optimizers().state_dict()[var_name])

In [ ]:
torch.save(model.state_dict(), os.path.join(saved_model_path, "final_saved_qa_model.pt"))

In [ ]:
model.to(device)

def generate(model, passage, question):
  source_encoding = tokenizer(
        question,
        passage,
        max_length=1024,
        padding="max_length",
        truncation="only_second",
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
    )
  
  # generated_token_ids = model.QAModel.generate(
  #     input_ids=source_encoding["input_ids"],
  #     attention_mask=source_encoding["attention_mask"],
  #     max_length=128,
  #     num_beams=5,
  #     repetition_penalty=2.5,
  #     length_penalty=1.0,
  #     early_stopping=True
  #   )
  generated_token_ids = model.model.generate(
      input_ids=source_encoding["input_ids"].to(device),
      attention_mask=source_encoding["attention_mask"].to(device),
      max_length=64,
      num_beams=5,
      repetition_penalty=2.5,
      length_penalty=1.0,
      early_stopping=True
    )
  
  preds = [
      tokenizer.decode(token_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
      for token_id in generated_token_ids
  ]

  return "".join(preds)

In [ ]:
sample_test_index = 110
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()
print(f"Time taken to generate response : {end_time - start_time}")

print("PASSAGE : {}\nQUESTION : {}\nRETRIEVED RESPONSE : {}\nGENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer))

Time taken to generate response : 2.4011449813842773
PASSAGE : create a dredging station. set the bowl of spiced buttermilk at your workstation. get out another wide bowl and set it next to the buttermilk bowl. measure 1/2 cup (60 g) of all-purpose flour into the empty bowl. set a plate next to the stove and lay a paper towel over it.
QUESTION : what is the nutritional value of buttermilk?
RETRIEVED RESPONSE : 1 cup of buttermilk has 152 calories, 8 g of protein, 12 g of carbohydrates, 12 g of sugar, 27 mg of cholesterol, and 8 g of fat.
GENERATED RESPONSE : 1 cup of buttermilk has 152 calories, 8 g of protein, 12 g of carbohydrates, 12 g of sugar, 27 mg of cholesterol, and 8 g of fat.


In [ ]:
sample_test_passage

'create a dredging station. set the bowl of spiced buttermilk at your workstation. get out another wide bowl and set it next to the buttermilk bowl. measure 1/2 cup (60 g) of all-purpose flour into the empty bowl. set a plate next to the stove and lay a paper towel over it.'

In [ ]:
sample_test_question

'what is the nutritional value of buttermilk?'

In [ ]:
sample_test_answer

'1 cup of buttermilk has 152 calories, 8 g of protein, 12 g of carbohydrates, 12 g of sugar, 27 mg of cholesterol, and 8 g of fat.'

In [ ]:
predicted_answer

'1 cup of buttermilk has 123 calories, 2 g of protein, 3 g of carbohydrates, and 4 g of sugar.'

In [ ]:
sample_test_question = 'what temperature to heat the oven ?'
predicted_answer = generate(model, sample_test_passage, sample_test_question)

In [ ]:
predicted_answer

'heat the oven to 375 degrees f (190 degrees c).'

# **SAMPLE GENERATED RESPONSES**

In [ ]:
import random

num_samples = 50
indices = random.sample(range(len(validation_df)), num_samples)

for sample_test_index in indices:
  sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
  sample_test_question = validation_df.iloc[sample_test_index]["question"]
  sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

  predicted_answer = generate(model, sample_test_passage, sample_test_question)

  # print("PASSAGE : {}\nQUESTION : {}\nRETRIEVED RESPONSE : {}\nGENERATED RESPONSE : {}\n\n".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer))
  print("PASSAGE : {}\nQUESTION : {}\nGENERATED RESPONSE : {}\n\n".format(sample_test_passage, sample_test_question, predicted_answer))

PASSAGE : eat the boneless bite of chicken wing. once you remove the 2 bones, you are left with a boneless chunk of meat. take 1-2 bites to eat your boneless wing. hold the chicken wing in both of your hands. before you take a bite, grip your chicken wing with your index finger and thumb. this way, you can easily tear into it and get the meat. do this for either the drumette or wingette.
QUESTION : how do i properly eat chicken in a professional setting?
GENERATED RESPONSE : if you are eating chicken in a restaurant, do not eat it with your fork.


PASSAGE : drain the water from the potatoes, then add the butter to the pot. carefully drain the potatoes into a colander. transfer the potatoes back into the hot pot, and then add 2 tablespoons (30 grams) of butter. mix the potatoes and butter together until the butter has completely melted. ghee is a great substitute for butter if you cant have dairy products. add the 3 cup (160 ml) of milk and mash the potatoes. pour the milk into the pot

# **Generate responses for Validation Set**

In [ ]:
responses = {}

latency = 0.0
passages, questions, answers, generated_answers = [], [], [], []
total = 0
for sample_test_index in range(len(validation_df)):
  if sample_test_index > 0 and sample_test_index % 100 == 0:
    print(f"{sample_test_index} responses of {len(validation_df)} generated.")

  passage = validation_df.iloc[sample_test_index]["passage"]
  question = validation_df.iloc[sample_test_index]["question"]
  answer = validation_df.iloc[sample_test_index]["answer"]

  if total > 20:
    break

  start_time = time.time()
  generated_answer = generate(model, passage, question)
  end_time = time.time()
  time_elapsed = end_time - start_time

  latency += time_elapsed

  passages.append(passage)
  questions.append(question)
  answers.append(answer)
  generated_answers.append(generated_answer)

  total += 1

print(f"Average response generation time : {latency / total}")

responses["passage"] = passages
responses["question"] = questions
responses["answer"] = answers
responses["generated_answer"] = generated_answers

responses_df = pd.DataFrame.from_dict(responses)
# responses_df.to_pickle(os.path.join(data_path, "validation_set_generated_responses_df.pickle"))
responses_df

Average response generation time : 2.9266112191336497


,passage,question,answer,generated_answer
0,print out your booking details. open the itine...,what does log in mean?,the verb log ing is usually defined as: to ent...,log in is usually defined as: to enter a websi...
1,place the shredded kale and garlic into the st...,what does collard greens mean?,collard greens is usually defined as: a form o...,collard greens is a form of the noun collard g...
2,allow the kiwi berries to ripen out of the fri...,what kind of food is kiwi?,"fruit, berry, item of produce, food item and v...","fruit, berry, item of produce, food item and v..."
3,prepare your beetroot. wash and scrub any dirt...,what does beetroot mean?,"beetroot is usually defined as: beet, the edib...","beetroot is usually defined as: beet, the edib..."
4,cinch the plastic bag around the top of the co...,what is the green onions?,according to wikipedia: scallions are vegetabl...,according to wikipedia: scallions are vegetabl...
5,"use a plastic disposable knife for clean, even...",what does knives mean?,"knives is usually defined as: plural of knife,...","knives is usually defined as: plural of knife,..."
6,allow the jars to cool before checking the sea...,what is the pears?,"pears is a form of the word pear. the noun, pe...","pears is a form of the word pear. the noun, pe..."
7,unplug your waffle iron and allow it to cool. ...,what is waffle made of?,waffles typically contain dough and they are m...,heres what i found on waffless wikipedia page:...
8,"after step 5, if you still have any meals unac...",what does peanut butter mean?,peanut butter is usually defined as: a paste m...,peanut butter is usually defined as: a paste m...
9,put your oats in a bowl. you can use rolled oa...,what kind of food is rolled oats?,rolled oats are oatmeal.,"manufactured food, breakfast cereal, breakfast..."


# **Testing flan-t5-xl inference**

In [ ]:
!pip install "transformers==4.26.0"
!pip install torch --extra-index-url https://download.pytorch.org/whl/cu116 --upgrade
!pip install accelerate
!pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 93.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.4
    Uninstalling transformers-4.27.4:
      Successfully uninstalled transformers-4.27.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
checkpoint = "/content/drive/MyDrive/Projects/Commonsense_Reasoning/checkpoint-5742"

In [ ]:
from transformers import (AutoModelForSeq2SeqLM,DataCollatorForSeq2Seq,AutoTokenizer,set_seed)

model_name = "google/flan-t5-xl"

In [ ]:
tokenizer_xl = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# model_xl = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto", load_in_8bit=True, return_dict=True)
model_xl = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto", return_dict=True)
model_xl.to(device)
model_xl.get_memory_footprint()

11399028736

In [ ]:
# sent = "What essence goes well with chocolate cake?"

def generate_xl(model, tokenizer, passage, question):
  input_text = "Generate answer for the question below :\n{" + passage + "\n" + question + "}\nAnswer:\n"
  inputs = tokenizer.encode(input_text, return_tensors="pt")
  generated_ids = model.generate(inputs, 
                                max_length=100,
                                num_beams=1,
                                repetition_penalty=2.5,
                                length_penalty=1.0,
                                early_stopping=True)
  # generated_ids = model.generate(inputs, 
  #                               max_length=100)
  generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return generated_text

# **Sample QA's with Context**

In [ ]:
import random

sample_test_index = random.randint(0, 5000)
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()

start_time_xl = time.time()
predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)
end_time_xl = time.time()

print(f"LARGE : Time taken to generate response : {end_time - start_time}")
print(f"XL : Time taken to generate response : {end_time_xl - start_time_xl}")

print("PASSAGE : {}\nQUESTION : {}\n\nRETRIEVED RESPONSE : {}\n\nLARGE GENERATED RESPONSE : {}\n\nXL GENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))

In [ ]:
import random

sample_test_index = random.randint(0, 5000)
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()

start_time_xl = time.time()
predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)
end_time_xl = time.time()

print(f"LARGE : Time taken to generate response : {end_time - start_time}")
print(f"XL : Time taken to generate response : {end_time_xl - start_time_xl}")

print("PASSAGE : {}\nQUESTION : {}\n\nRETRIEVED RESPONSE : {}\n\nLARGE GENERATED RESPONSE : {}\n\nXL GENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))

LARGE : Time taken to generate response : 3.2038116455078125
XL : Time taken to generate response : 6.73159122467041
PASSAGE : spread the peanuts on the sheet and bake them for 20 to 25 minutes. transfer the toffee-coated peanuts to the greased baking dish and spread them evenly across the pan. put it in the preheated oven and bake the peanuts until they are a rich, dark color. this should take about 20 to 25 minutes. you may want to stir the peanuts once during the baking time to ensure that they cook evenly.
QUESTION : what is the meaning of peanuts?

RETRIEVED RESPONSE : peanuts is a form of the word peanut. as a noun, peanut is usually defined as: the pod or the enclosed edible seed of the plant, arachis hypogaea, of the legume family: the pod is forced underground in growing, where it ripens. as an adjective, peanut is usually defined as: of or relating to the peanut or peanuts. for more, ask me to give you more definitions for peanut.

LARGE GENERATED RESPONSE : peanuts is a form

In [ ]:
import random

sample_test_index = random.randint(0, 5000)
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()

start_time_xl = time.time()
predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)
end_time_xl = time.time()

print(f"LARGE : Time taken to generate response : {end_time - start_time}")
print(f"XL : Time taken to generate response : {end_time_xl - start_time_xl}")

print("PASSAGE : {}\nQUESTION : {}\n\nRETRIEVED RESPONSE : {}\n\nLARGE GENERATED RESPONSE : {}\n\nXL GENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))

LARGE : Time taken to generate response : 1.1315419673919678
XL : Time taken to generate response : 1.530712366104126
PASSAGE : add the juice and yogurt. if you would like your smoothie to be sweet, add in the honey. if you do not have any agave nectar, you can use another sweetener, such as honey, stevia, or sugar. top it off with ice. if you are using frozen bananas, then you can use less ice or skip it entirely.
QUESTION : do i have to use ice, or is it optional?

RETRIEVED RESPONSE : the ice is there to help thicken and chill your smoothie. if you do not care about thickness, you can skip the ice; if you want your smoothie to be cold, stick it in the fridge for a few minutes to chill. you can also use other things instead of ice, such as frozen bananas or ice cream (ie: chocolate or vanilla).

LARGE GENERATED RESPONSE : if you do not have ice, then you do not need to use it.

XL GENERATED RESPONSE : ice is usually recommended for cold smoothies, but it can also be used as part of a

In [ ]:
import random

sample_test_index = random.randint(0, 5000)
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()

start_time_xl = time.time()
predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)
end_time_xl = time.time()

print(f"LARGE : Time taken to generate response : {end_time - start_time}")
print(f"XL : Time taken to generate response : {end_time_xl - start_time_xl}")

print("PASSAGE : {}\nQUESTION : {}\n\nRETRIEVED RESPONSE : {}\n\nLARGE GENERATED RESPONSE : {}\n\nXL GENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))

LARGE : Time taken to generate response : 1.5035009384155273
XL : Time taken to generate response : 4.486846923828125
PASSAGE : stop watering around october. when october hits, your watering duties are over. you can carefully resume a light watering in november. if it is dry where you live, feel free to place the pot over a tray of moist pebbles. this is all to set up the bloom happening around christmas. the only other time you should cease watering is after the plant blooms. at this time, cease watering for about 6 weeks to allow the plant to rest. new growth will still appear  and that is when watering should be resumed. watch out for bud drop. one of the most frustrating things that can happen to christmas cactus is after the flower buds have developed they drop off the plant. bud drop can be caused by several different conditions. usually it is because of over-watering, but it can also be due to a lack of humidity or insufficient light. if this starts happening, water the plant le

In [ ]:
import random

sample_test_index = random.randint(0, 5000)
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()

start_time_xl = time.time()
predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)
end_time_xl = time.time()

print(f"LARGE : Time taken to generate response : {end_time - start_time}")
print(f"XL : Time taken to generate response : {end_time_xl - start_time_xl}")

print("PASSAGE : {}\nQUESTION : {}\n\nRETRIEVED RESPONSE : {}\n\nLARGE GENERATED RESPONSE : {}\n\nXL GENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))

LARGE : Time taken to generate response : 0.9813899993896484
XL : Time taken to generate response : 0.9243316650390625
PASSAGE : if you are planning to keep the tiles and removing just the grout, cover the edges of each tile with painters tape. cover the surfaces of the appliances you are planning to keep. as the grout is removed, flying debris can cause damage. drape a large blanket or drop cloth over the appliances, making sure they are fully covered. if necessary, use tape to secure the edges of the protective covering and make sure to cover drains completely. don protective clothing before starting the grout removal process. this should include safety glasses, dust mask and cut resistant gloves. for the purpose of comfort, consider wearing knee pads, as it can take a while to remove grout. make sure to wear pants and boots as well.
QUESTION : does the area being grouted need to be completely dry?

RETRIEVED RESPONSE : the tile does not have to be completely dry, a little moisture w

In [ ]:
import random

sample_test_index = random.randint(0, 5000)
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()

start_time_xl = time.time()
predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)
end_time_xl = time.time()

print(f"LARGE : Time taken to generate response : {end_time - start_time}")
print(f"XL : Time taken to generate response : {end_time_xl - start_time_xl}")

print("PASSAGE : {}\nQUESTION : {}\n\nRETRIEVED RESPONSE : {}\n\nLARGE GENERATED RESPONSE : {}\n\nXL GENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))

LARGE : Time taken to generate response : 0.8933181762695312
XL : Time taken to generate response : 0.551295280456543
PASSAGE : cut the kevlar fabric into a strip. use the scissors to cut a strip of kevlar fabric thats about 4 inches (10 cm) wide and 2 feet (61 cm) long. you can buy kevlar fabric at some home stores, department and hardware stores, fabric stores, or online. kevlar is a durable synthetic fabric thats made from plastic. however, its flame-resistant and doesnt melt, making it ideal for torches. kevlar is often used by fire jugglers and fire poi performers.
QUESTION : what is poi made of?

RETRIEVED RESPONSE : poi is made of taro.

LARGE GENERATED RESPONSE : poi is made from glutinous rice.

XL GENERATED RESPONSE : poi is made of rice.


In [ ]:
import random

sample_test_index = random.randint(0, 5000)
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()

start_time_xl = time.time()
predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)
end_time_xl = time.time()

print(f"LARGE : Time taken to generate response : {end_time - start_time}")
print(f"XL : Time taken to generate response : {end_time_xl - start_time_xl}")

print("PASSAGE : {}\nQUESTION : {}\n\nRETRIEVED RESPONSE : {}\n\nLARGE GENERATED RESPONSE : {}\n\nXL GENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))

LARGE : Time taken to generate response : 0.6894574165344238
XL : Time taken to generate response : 0.669804573059082
PASSAGE : add the bean sprouts and cook for 1 minute and 30 seconds. the water will die down when you add the bean sprouts, so let it return to a boil first. once it is boiling again, set the timer for 1 minute and 30 seconds. be sure to rinse the bean sprouts with filtered water first. you can also remove the brown threads from the roots, but you do not have to.
QUESTION : what kind of food is bean sprouts?

RETRIEVED RESPONSE : mung bean sprouts are sprouts.

LARGE GENERATED RESPONSE : mung bean sprouts are sprouts.

XL GENERATED RESPONSE : mung bean sprouts are legume.


In [ ]:
import random

sample_test_index = random.randint(0, 5000)
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()

start_time_xl = time.time()
predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)
end_time_xl = time.time()

print(f"LARGE : Time taken to generate response : {end_time - start_time}")
print(f"XL : Time taken to generate response : {end_time_xl - start_time_xl}")

print("PASSAGE : {}\nQUESTION : {}\n\nRETRIEVED RESPONSE : {}\n\nLARGE GENERATED RESPONSE : {}\n\nXL GENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))

LARGE : Time taken to generate response : 0.5221436023712158
XL : Time taken to generate response : 0.5410304069519043
PASSAGE : puree the peaches in a blender or food processor. peel and slice the peaches first, then put them into a blender or food processor. puree them until you get the consistency you want. the longer you puree them for, the smoother your ice cream will be. if you cannot get any fresh peaches, you can use 4 cups (900 grams) of frozen, unsweetened peach slices instead. let the peaches thaw first; this will make it easier to mix them into the custard.
QUESTION : how much is 4 cups?

RETRIEVED RESPONSE : 4 cups is 1 quart.

LARGE GENERATED RESPONSE : 4 cups is 1 quart.

XL GENERATED RESPONSE : 4 cups is 1 quart.


In [ ]:
import random

sample_test_index = random.randint(0, 5000)
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()

start_time_xl = time.time()
predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)
end_time_xl = time.time()

print(f"LARGE : Time taken to generate response : {end_time - start_time}")
print(f"XL : Time taken to generate response : {end_time_xl - start_time_xl}")

print("PASSAGE : {}\nQUESTION : {}\n\nRETRIEVED RESPONSE : {}\n\nLARGE GENERATED RESPONSE : {}\n\nXL GENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))

LARGE : Time taken to generate response : 3.0810768604278564
XL : Time taken to generate response : 5.38700008392334
PASSAGE : add a small amount of mustard to a small bowl. you should add a dollop of mustard that is about the size of your fingertip. mustard acts as an emulsifying agent--when two liquids dont mix, like water and oil, mustard breaks them down and helps them to mix. you can also add a fingertip-sized dollop of mayonnaise to the bowl. mayo is also an emulsifier and adds a nice creamy texture to the vinaigrette. however, for a lower-calorie vinaigrette, skip the mayo.
QUESTION : what is the meaning of mustard?

RETRIEVED RESPONSE : mustard is usually defined as: a pungent powder or paste prepared from the seed of the mustard plant, used as a food seasoning or condiment, and medicinally in plasters, poultices, etc. for more, ask me to give you more definitions for mustard.

LARGE GENERATED RESPONSE : mustard is usually defined as: a pungent powder or paste prepared from the

In [ ]:
import random

sample_test_index = random.randint(0, 5000)
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()

start_time_xl = time.time()
predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)
end_time_xl = time.time()

print(f"LARGE : Time taken to generate response : {end_time - start_time}")
print(f"XL : Time taken to generate response : {end_time_xl - start_time_xl}")

print("PASSAGE : {}\nQUESTION : {}\n\nRETRIEVED RESPONSE : {}\n\nLARGE GENERATED RESPONSE : {}\n\nXL GENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))

LARGE : Time taken to generate response : 0.5765705108642578
XL : Time taken to generate response : 0.6363348960876465
PASSAGE : tear a 2 x 2 inch (5 x 5cm) square of duct tape. it does not have to be exact, but it should be approximately the size mentioned. however, if you want all of your petals to be the same size, feel free to use a ruler to measure 2 inches (5 cm) of width and height. then, rip or cut off the piece.
QUESTION : what are 2 inches?

RETRIEVED RESPONSE : 2 inches is 0.1667 feet.

LARGE GENERATED RESPONSE : 2 inches is 0.1667 feet.

XL GENERATED RESPONSE : 2 inches is 0.1667 feet.


In [ ]:
import random

sample_test_index = random.randint(0, 5000)
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()

start_time_xl = time.time()
predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)
end_time_xl = time.time()

print(f"LARGE : Time taken to generate response : {end_time - start_time}")
print(f"XL : Time taken to generate response : {end_time_xl - start_time_xl}")

print("PASSAGE : {}\nQUESTION : {}\n\nRETRIEVED RESPONSE : {}\n\nLARGE GENERATED RESPONSE : {}\n\nXL GENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))

LARGE : Time taken to generate response : 1.185556411743164
XL : Time taken to generate response : 0.7402958869934082
PASSAGE : remember that school buses stop at all railroad crossings. be prepared to stop if youre behind one and approaching a crossingwhether or not a train is on its way. pass a school bus on its left hand side only. the right hand side is where all loading and unloading happens. even if the school bus is in a left hand lane, do not pass on the right.
QUESTION : what is the rule in texas with a road with a center lane for turning?

RETRIEVED RESPONSE : only pull into the turn lane right before you are about to turn, but soon enough that you can stop safely for the turn. if you miss the turn, do not drive in the turn lane. oncoming traffic uses the same lane and may collide with you head-on. get back into the regular lanes as soon as possible, and the next time you can turn, move back into the turn lane after you have checked and it is clear.

LARGE GENERATED RESPONSE 

In [ ]:
import random

sample_test_index = random.randint(0, 5000)
sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
sample_test_question = validation_df.iloc[sample_test_index]["question"]
sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

start_time = time.time()
predicted_answer = generate(model, sample_test_passage, sample_test_question)
end_time = time.time()

start_time_xl = time.time()
predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)
end_time_xl = time.time()

print(f"LARGE : Time taken to generate response : {end_time - start_time}")
print(f"XL : Time taken to generate response : {end_time_xl - start_time_xl}")

print("PASSAGE : {}\nQUESTION : {}\n\nRETRIEVED RESPONSE : {}\n\nLARGE GENERATED RESPONSE : {}\n\nXL GENERATED RESPONSE : {}".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))

LARGE : Time taken to generate response : 1.287684679031372
XL : Time taken to generate response : 1.4513428211212158
PASSAGE : make main course meals using the leftover ham. some examples include: make rice and ham with sweet and sour sauce. make ham fritters. find a favorite fritter recipe and add ham pieces to it. fritter recipes that combine the ingredients with beans or lentils are a good choice. make ham and leek rolls. add pieces of ham to a frittata or sprinkle over a pizza. make quesadillas with ham as part of the filling. toss ham into any casserole, savory bake or stew you are making, for added flavor. make scalloped potatoes with ham.
QUESTION : what kind of food is leek?

RETRIEVED RESPONSE : modern british cuisine, ethnic food, european food, welsh cuisine, edible and british food.

LARGE GENERATED RESPONSE : vegetable, item of produce, edible, food item and vegetarian food item.

XL GENERATED RESPONSE : modern british cuisine, ethnic food, european food, welsh stew and e

# **Generating responses from flan-t5-xl checkpoint**

In [ ]:
import random

num_samples = 50
indices = random.sample(range(len(validation_df)), num_samples)

for sample_test_index in indices:
  sample_test_passage = validation_df.iloc[sample_test_index]["passage"]
  sample_test_question = validation_df.iloc[sample_test_index]["question"]
  sample_test_answer = validation_df.iloc[sample_test_index]["answer"]

  predicted_answer = generate(model, sample_test_passage, sample_test_question)
  predicted_answer_xl = generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)

  print("PASSAGE : {}\nQUESTION : {}\nRETRIEVED RESPONSE : {}\nLARGE GENERATED RESPONSE : {}\nXL GENERATED RESPONSE : {}\n\n".format(sample_test_passage, sample_test_question, sample_test_answer, predicted_answer, predicted_answer_xl))
  # print("PASSAGE : {}\nQUESTION : {}\nGENERATED RESPONSE : {}\n\n".format(sample_test_passage, sample_test_question, predicted_answer))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1359: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


PASSAGE : display 2-3 potted plants around the den. potted plants are great for adding a fresh and vibrant feeling to a room. choose from small potted plants to display on a table or shelf, or a larger plant that stands out on the floor. fake plants are also a good option if you do not have time to care for a live plant. include a tv for watching movies. watching movies as a family is one of the best uses for a den. this makes the space especially inviting for children, and will build fun family memories. a tv mounted on the wall will keep it out of the way, and less of a focal point in the room. you can install wall shelves around the tv, and fill them with books, plants, and games too.
QUESTION : what can i put in a small childrens den?
RETRIEVED RESPONSE : large, colorful toys, a cute nightlight, and maybe even a small tent (my 2-year-old loves hers!).
LARGE GENERATED RESPONSE : you can put a bed, a dresser, and some books on shelves.
XL GENERATED RESPONSE : can you put a bean bag c

In [ ]:
responses = {}

latency = 0.0
passages, questions, answers, generated_answers = [], [], [], []
total = 0
for sample_test_index in range(len(validation_df)):

  # if total > 20:
  #   break

  if sample_test_index > 0 and sample_test_index % 100 == 0:
    print(f"{sample_test_index} responses of {len(validation_df)} generated.")

  passage = validation_df.iloc[sample_test_index]["passage"]
  question = validation_df.iloc[sample_test_index]["question"]
  answer = validation_df.iloc[sample_test_index]["answer"]

  start_time = time.time()
  generated_answer = generate_xl(model_xl, tokenizer_xl, passage, question)
  end_time = time.time()
  time_elapsed = end_time - start_time

  latency += time_elapsed

  passages.append(passage)
  questions.append(question)
  answers.append(answer)
  generated_answers.append(generated_answer)

  total += 1

print(f"Average response generation time : {latency / total}")

responses["passage"] = passages
responses["question"] = questions
responses["answer"] = answers
responses["generated_answer"] = generated_answers

responses_df = pd.DataFrame.from_dict(responses)
responses_df.to_pickle(os.path.join(data_path, "validation_set_generated_responses_xl_df.pickle"))
responses_df

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1359: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


100 responses of 5098 generated.
200 responses of 5098 generated.
300 responses of 5098 generated.
400 responses of 5098 generated.
500 responses of 5098 generated.
600 responses of 5098 generated.
700 responses of 5098 generated.
800 responses of 5098 generated.
900 responses of 5098 generated.
1000 responses of 5098 generated.
1100 responses of 5098 generated.
1200 responses of 5098 generated.
1300 responses of 5098 generated.
1400 responses of 5098 generated.
1500 responses of 5098 generated.
1600 responses of 5098 generated.
1700 responses of 5098 generated.
1800 responses of 5098 generated.


Token indices sequence length is longer than the specified maximum sequence length for this model (647 > 512). Running this sequence through the model will result in indexing errors


1900 responses of 5098 generated.
2000 responses of 5098 generated.
2100 responses of 5098 generated.
2200 responses of 5098 generated.
2300 responses of 5098 generated.
2400 responses of 5098 generated.
2500 responses of 5098 generated.
2600 responses of 5098 generated.
2700 responses of 5098 generated.
2800 responses of 5098 generated.
2900 responses of 5098 generated.
3000 responses of 5098 generated.
3100 responses of 5098 generated.
3200 responses of 5098 generated.
3300 responses of 5098 generated.
3400 responses of 5098 generated.
3500 responses of 5098 generated.
3600 responses of 5098 generated.
3700 responses of 5098 generated.
3800 responses of 5098 generated.
3900 responses of 5098 generated.
4000 responses of 5098 generated.
4100 responses of 5098 generated.
4200 responses of 5098 generated.
4300 responses of 5098 generated.
4400 responses of 5098 generated.
4500 responses of 5098 generated.
4600 responses of 5098 generated.
4700 responses of 5098 generated.
4800 responses

,passage,question,answer,generated_answer
0,print out your booking details. open the itine...,what does log in mean?,the verb log ing is usually defined as: to ent...,the verb log ing is usually defined as: to ent...
1,place the shredded kale and garlic into the st...,what does collard greens mean?,collard greens is usually defined as: a form o...,collard greens is usually defined as: a form o...
2,allow the kiwi berries to ripen out of the fri...,what kind of food is kiwi?,"fruit, berry, item of produce, food item and v...","fruit, berry, item of produce, food item and v..."
3,prepare your beetroot. wash and scrub any dirt...,what does beetroot mean?,"beetroot is usually defined as: beet, the edib...",beetroot is usually defined as: any of various...
4,cinch the plastic bag around the top of the co...,what is the green onions?,according to wikipedia: scallions are vegetabl...,according to wikipedia: the green onion is an ...
...,...,...,...,...
5093,develop a regional theme. one method of determ...,what cheese is the netherlands famous for making?,"one, cottage cheese contains fewer carbohydrat...","one, cottage cheese contains fewer carbohydrat..."
5094,serve the macaroni and cheese immediately or s...,which president brought mac and cheese to amer...,"yes. it will require more butter or margarine,...",this depends on how much pasta you made. if it...
5095,follow another simple meatball recipe. by comb...,is spaghetti and meatballs an italian dish?,"yes, but remember that if you do it that way, ...","yes, you can."
5096,cool the seeds. once the seeds are done toasti...,which country grows the most sesame seeds in t...,put your sesame seeds in a strainer and then f...,sesame seeds are crunchy with a slightly bitte...


# **Sample QA's without Context**

In [ ]:
sample_test_passage, sample_test_question = "", "Why is pre-heating oven important?"
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : pre-heating an oven is a necessary step in the process of heating. there are three types of pre-heating: electric, gas and natural gas.


XL : for baking, it is important to heat the oven up before you put any ingredients in. this will help get everything out of your way so there are no mix-ups when trying to bake something different than what was intended!


In [ ]:
sample_test_passage, sample_test_question = "", "In apple pie, how thin should the apples be cut?"
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : apple pie crust should be about 1/2 inch thick.


XL : the slices should be about 1/8th of an inch.


In [ ]:
sample_test_passage, sample_test_question = "", "Can I use cinnamon powder if I'm out of cinnamon?"
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : yes, you can use it as a substitute.


XL : yes. 1 teaspoon of cinnamon powder will work in place as a substitute for the missing ingredient(s).


In [ ]:
sample_test_passage, sample_test_question = "", "Can I go shopping while my oven is still on?"
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : yes, you can go shopping while your oven is still on.


XL : you should not do that because your oven will start to heat up the room. leaving it on while shopping could cause an accident or fire.


In [ ]:
sample_test_passage, sample_test_question = "", "How many oranges do I need to make a glass of orange juice?"
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : it depends on how much you want your juice to taste like. i would say start with around 1/8th of a orange and work your way up from there.


XL : it depends on how much you want. a typical glass of orange juice is 12 oz.


In [ ]:
sample_test_passage, sample_test_question = "", "How many apples do I need to make an apple pie?"
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : an apple pie is made with 24 apples.


XL : you will need at least 8 apples to make a pie.


In [ ]:
sample_test_passage, sample_test_question = "", "How to fry chicken properly?"
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : fry the chicken until it is no longer pink.


XL : start by thawing the chicken in warm water for about 15 minutes. pat it dry with paper towels, then fry it in a hot wok or skillet with cooking oil for about 3 minutes.


In [ ]:
sample_test_passage, sample_test_question = "", "How to open wine bottle without cork opener?"
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : wine bottle openers are used to remove corks from wine bottles. you will need a screwdriver or jewelers screwdriver to open the cork.


XL : use a wine bottle stopper; place it in the wine bottle neck and twist the stopper until it disengages. use a corkscrew to remove the cork.


In [ ]:
sample_test_passage, sample_test_question = "", "Suggest some good Thanksgiving recipes."
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : heres what i found on Thanksgivings wikipedia page: thanksgiving is a federal holiday in the united states. there are various recipes including stuffing, corned beef, mashed potatoes, gravy, and more.


XL : heres something i found on reference.com: traditional thanksgiving dinner includes roasting a turkey in a large oven, stuffing it with cranberries and vegetables, and serving it with mashed potatoes, stuffing, and gravy. other traditional dishes include stuffing a goose, roasting a pork chop, making cranberry sauce, and baking a pumpkin. for more ideas, see wikihow is article on making traditional thanksgiving dinner


In [ ]:
sample_test_passage, sample_test_question = "", "I love watching movies. What movies do you like to watch?"
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : that is a question i would like to ask you.


XL : i like to watch action, comedy, and romantic comedies.


***Quantity based response***

In [ ]:
sample_test_passage, sample_test_question = "", "How many eggs do I need to bake a 2 pound cake?"
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : 2 pounds is 32 ounces.


XL : for a 2 pound cake, you will need 2 eggs.


In [ ]:
sample_test_passage, sample_test_question = "", "How many eggs do I need to bake a half kilo cake?"
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : it depends on the size of your cake. a half kilo cake would need around 12 eggs.


XL : you will need 12 eggs.


In [ ]:
sample_test_passage, sample_test_question = "", "How many eggs do I need to bake a one kilo cake?"
print("LARGE : {}".format(generate(model, sample_test_passage, sample_test_question)))
print("\n\nXL : {}".format(generate_xl(model_xl, tokenizer_xl, sample_test_passage, sample_test_question)))

LARGE : one kilo cake will need about 12 eggs.


XL : for a one kilo cake, you will need at least two eggs.
